In [1]:
library(glmnet)

Loading required package: Matrix
Loading required package: foreach
Loaded glmnet 2.0-16



In [2]:
set.seed(43)

In [3]:
n <- 1000 # Number of observations
p <- 5000 # Number of predictors included in model
real_p <- 15 # Number of true predictors

In [4]:
x = matrix(rnorm(n*p),nrow=n,ncol=p)

In [7]:
y <- apply(x[,1:real_p],1,sum)+rnorm(n)

In [8]:
train_rows <- sample(1:n,0.66*n)

In [12]:
X.train <- x[train_rows,]

In [13]:
X.test <- x[-train_rows,]

In [14]:
y.train <- y[train_rows]

In [15]:
y_test <- y[-train_rows]

In [ ]:
## s = is the "size" of the penalty that we want to use, and thus, corresponds to lambda. 

# RIDGE REGRESSION

In [16]:
alpha0.fit <- cv.glmnet(X.train,y.train,type.measure="mse",alpha=0,family="gaussian")

In [17]:
alpha0.predicted <- predict(alpha0.fit , s=alpha0.fit$lambda.1se,newx=X.test)

In [18]:
mean((y_test - alpha0.predicted)^2)

[1] 15.16141

# LASSO REGRESSION

In [19]:
alpha1.fit <- cv.glmnet(X.train,y.train,type.measure="mse",alpha=1,family="gaussian")

In [20]:
alpha1.predicted <- predict(alpha1.fit , s=alpha1.fit$lambda.1se,newx=X.test)

In [21]:
mean((y_test - alpha1.predicted)^2)

[1] 1.375312

# ELASTIC NET

In [22]:
alpha0.5.fit <- cv.glmnet(X.train,y.train,type.measure="mse",alpha=0.5,family="gaussian")

In [23]:
alpha0.5.predicted <- predict(alpha0.5.fit , s=alpha1.fit$lambda.1se,newx=X.test)

In [24]:
mean((y_test - alpha0.5.predicted)^2)

[1] 1.380705

In [36]:
##the best thing to do is just try a bunch of different
## values for alpha rather than guess which one will be best.
## The following loop uses 10-fold Cross Validation to determine the
## optimal value for lambda for alpha = 0, 0.1, ... , 0.9, 1.0
## using the Training dataset.

In [25]:
list.of.fits <- list()

In [29]:
for (i in 1:10) {
    fit.name <- paste0("alpha",i/10)
    
    list.of.fits[[fit.name]] <- cv.glmnet(X.train,y.train,type.measure="mse",alpha=i/10,family="gaussian")
}

In [31]:
result <- data.frame()

In [33]:
for (i in 1:10) {
    fit.name <- paste0("alpha",i/10)
    
    predicted <- predict( list.of.fits[[fit.name]], s=list.of.fits[[fit.name]]$lambda.1se,newx=X.test)
    
    mse <- mean((y_test-predicted)^2)
    
    temp <- data.frame(alpha=i/10,mse=mse,fit.name=fit.name)
    
    result <- rbind(result,temp)
    
}

In [34]:
result

alpha,mse,fit.name
0.1,2.365872,alpha0.1
0.2,1.681342,alpha0.2
0.3,1.516342,alpha0.3
0.4,1.488059,alpha0.4
0.5,1.472989,alpha0.5
0.6,1.421112,alpha0.6
0.7,1.385212,alpha0.7
0.8,1.398067,alpha0.8
0.9,1.358121,alpha0.9
1.0,1.411247,alpha1
